In [ ]:
# % pip install "zenml[server]"
# !zenml integration install sklearn -y
# %pip install pyparsing==2.4.2

# import IPython
# IPython.Application.instance().kernel.do_shutdown(restart=True)

In [1]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC 
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

def train_test() -> None:
  digits = load_digits()
  data = digits.images.reshape((len(digits.images), -1))
  X_train, X_test, y_train, y_test = train_test_split(
    data, digits.target, test_size=0.2, shuffle=False
  )
  model = SVC(gamma=0.001)
  model.fit(X_train, y_train)
  test_acc = model.score(X_test, y_test)
  print(f"Test accuray: {test_acc}")

train_test()

Test accuray: 0.9583333333333334


In [11]:
from zenml import step
from typing_extensions import Annotated
import pandas as pd 
from typing import Tuple

@step
def importer() -> Tuple[ 
  Annotated[np.ndarray, "X_train"], 
  Annotated[np.ndarray, "X_test"], 
  Annotated[np.ndarray, "y_train"], 
  Annotated[np.ndarray, "y_test"]
]:
  digits = load_digits()
  data = digits.images.reshape((len(digits.images), -1))
  X_train, X_test, y_train, y_test = train_test_split(
    data, digits.target, test_size=0.2, shuffle=False
  )
  return X_train, X_test, y_train, y_test

@step
def svc_trainer(X_train: np.ndarray, y_train: np.ndarray) -> ClassifierMixin:
  model = SVC(gamma=0.001)
  model.fit(X_train, y_train)
  return model

@step
def evaluator(X_test: np.ndarray, y_test: np.ndarray, model: ClassifierMixin) -> float:
  test_acc = model.score(X_test, y_test)
  print(f"Test accuracy: {test_acc}")
  return test_acc

In [12]:
from zenml import pipeline

@pipeline
def digits_pipeline():
  X_train, X_test, y_train, y_test = importer()
  model = svc_trainer(X_train=X_train, y_train=y_train)
  evaluator(X_test=X_test, y_test=y_test, model=model)

In [13]:
digits_svc_pipeline = digits_pipeline()

Initiating a new run for the pipeline: digits_pipeline.
Registered new version: (version 2).
Executing a new run.
Using user: default
Using stack: default
  artifact_store: default
  orchestrator: default
Step importer has started.
Skipping visualization of numpy array because matplotlib is not installed. To install matplotlib, run pip install matplotlib.
Skipping visualization of numpy array because matplotlib is not installed. To install matplotlib, run pip install matplotlib.
Step importer has finished in 4.404s.
Step svc_trainer has started.
Step svc_trainer has finished in 1.015s.
Step evaluator has started.
Test accuracy: 0.9583333333333334
Step evaluator has finished in 1.153s.
Run digits_pipeline-2023_12_09-18_48_30_122487 has finished in 6.954s.
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.
